<a href="https://colab.research.google.com/github/semral3021/SamralThesis/blob/main/Lab1_6_RAG_Semantic_Fiction_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 1.6 — Retrieval-Augmented Generation (RAG)
This notebook extends my thesis-based end-to-end semantic analysis agent by adding retrieval-based context using ChromaDB.


In [1]:
!pip install -U google-genai chromadb sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 9.2 MB/s eta

In [2]:
from google.colab import userdata

GEMINI_KEY = userdata.get("GEMINI_KEY")
print("Key loaded:", GEMINI_KEY is not None)


Key loaded: True


In [3]:
import google.genai as genai
client = genai.Client(api_key=GEMINI_KEY)


In [6]:
import random

characters = ["Evelyn", "Marcus", "Hale", "Arin", "Mira", "Lena", "Dr. Solis", "Captain Rhea"]
locations = ["control room", "hangar", "abandoned station", "old chamber", "research lab", "colony ruins"]
themes = ["responsibility", "betrayal", "fear", "courage", "uncertainty", "hope", "danger", "loyalty"]
tones = ["tense", "mysterious", "hopeful", "fearful", "urgent", "calm", "melancholic"]
roles = ["inciting incident", "moral dilemma", "turning point", "climax", "introduction"]

dataset = []

for i in range(1000):
    c1 = random.choice(characters)
    c2 = random.choice([c for c in characters if c != c1])
    loc = random.choice(locations)
    theme = random.choice(themes)
    tone = random.choice(tones)
    role = random.choice(roles)

    x = f"{c1} encountered {c2} in the {loc}, where tensions rose and the truth began to unfold."
    y = {
        "characters": [c1, c2],
        "entities": [loc],
        "themes": [theme],
        "sentiment": "neutral",
        "tone": tone,
        "narrative_role": role,
        "character_dynamics": f"{c1} and {c2} experience conflict related to {theme}.",
        "keywords": [c1, c2, loc, theme]
    }

    dataset.append((x, y))

len(dataset)



1000

In [11]:
chroma_client.delete_collection("fiction_examples")
collection = chroma_client.create_collection("fiction_examples")


In [12]:
collection = chroma_client.create_collection("fiction_examples_v2")


In [13]:
import chromadb
from sentence_transformers import SentenceTransformer

chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection("fiction_examples")

embedder = SentenceTransformer("all-MiniLM-L6-v2")


In [15]:
import json

for idx, (x, y) in enumerate(dataset):
    emb = embedder.encode(x).tolist()

    collection.add(
        ids=[f"id_{idx}"],
        documents=[x],
        metadatas=[{"y": json.dumps(y)}],   # store y as a string
        embeddings=[emb]
    )

print("Inserted", len(dataset), "examples into ChromaDB.")


Inserted 1000 examples into ChromaDB.


In [17]:
print(result)


X: Commander Anya Sharma traced the dust motes dancing in the faint, alien sunlight filtering through her habitat's viewport. Mars was a distant memory. This nameless planet, Epsilon-7, stretched out in a crimson void. Suddenly, the comms panel, usually dormant, crackled to life. A voice, clear and melodic, spoke a language she didn't recognize, yet its cadence was undeniably inquisitive. She froze, her hand hovering over the 'reply' button, a mix of terror and profound wonder gripping her.
y:
{
  "characters": ["Commander Anya Sharma", "Unknown Voice"],
  "entities": ["Epsilon-7", "habitat", "comms panel", "dust motes", "alien sunlight"],
  "themes": ["First Contact", "Isolation", "Discovery", "The Unknown", "Exploration"],
  "sentiment": "mixed",
  "tone": "mysterious, suspenseful, awe-inspiring, solitary",
  "narrative_role": "inciting incident, rising action, world-building, character development",
  "character_dynamics": "isolated protagonist facing a potential non-human interacti

In [18]:
def retrieve_similar(query):
    q_emb = embedder.encode(query).tolist()
    results = collection.query(query_embeddings=[q_emb], n_results=3)

    retrieved = []
    for i in range(len(results["documents"][0])):
        x = results["documents"][0][i]
        y = json.loads(results["metadatas"][0][i]["y"])
        retrieved.append((x, y))

    return retrieved


In [19]:
output = retrieve_similar("Evelyn entered the lab as alarms echoed.")
output


[('Hale encountered Evelyn in the research lab, where tensions rose and the truth began to unfold.',
  {'characters': ['Hale', 'Evelyn'],
   'entities': ['research lab'],
   'themes': ['responsibility'],
   'sentiment': 'neutral',
   'tone': 'calm',
   'narrative_role': 'inciting incident',
   'character_dynamics': 'Hale and Evelyn experience conflict related to responsibility.',
   'keywords': ['Hale', 'Evelyn', 'research lab', 'responsibility']}),
 ('Hale encountered Evelyn in the research lab, where tensions rose and the truth began to unfold.',
  {'characters': ['Hale', 'Evelyn'],
   'entities': ['research lab'],
   'themes': ['uncertainty'],
   'sentiment': 'neutral',
   'tone': 'urgent',
   'narrative_role': 'inciting incident',
   'character_dynamics': 'Hale and Evelyn experience conflict related to uncertainty.',
   'keywords': ['Hale', 'Evelyn', 'research lab', 'uncertainty']}),
 ('Hale encountered Evelyn in the research lab, where tensions rose and the truth began to unfold.'

In [20]:
def build_rag_prompt(new_input):
    retrieved = retrieve_similar(new_input)

    examples_text = ""
    for i, (x, y) in enumerate(retrieved, 1):
        examples_text += f"""
Example {i}:
X: {x}
y: {json.dumps(y, indent=2)}
"""

    final_prompt = f"""
Use the following retrieved examples to guide your semantic analysis.

{examples_text}

Now analyze this input:
X: {new_input}

Return only y in the same JSON-like structure.
"""

    return final_prompt


In [23]:
new_X = """Evelyn rushed into the damaged hangar,
searching for answers as the station's alarms echoed."""


In [24]:
aug_prompt = build_rag_prompt(new_X)

response = client.models.generate_content(
    model="models/gemini-2.5-flash",
    contents=aug_prompt
)

print(response.text)


```json
{
  "characters": [
    "Evelyn"
  ],
  "entities": [
    "hangar",
    "station",
    "alarms"
  ],
  "themes": [
    "fear"
  ],
  "sentiment": "neutral",
  "tone": "fearful",
  "narrative_role": "turning point",
  "character_dynamics": "Evelyn experiences conflict related to fear.",
  "keywords": [
    "Evelyn",
    "hangar",
    "alarms",
    "fear"
  ]
}
```
